In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import nltk
# nltk.download()

import textstat

In [5]:
import pickle
import json, csv
import os

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize

In [ ]:
movie_details = pickle.load(open('movie_details_combined.pkl', 'rb'))

In [ ]:
missing_synopsis = []

for idx, movie_detail in enumerate(movie_details):
    if len(movie_detail['plot_synopsis']) == 0:
        missing_synopsis.append(movie_detail)

print('Number of Movies: ' + str(len(movie_details)) + 
      '\nMovies Missing Plot Synopsis: ' + str(len(missing_synopsis)))

movies_with_synopsis = [] #use this one in loop

for movie_detail in movie_details:
    if movie_detail in missing_synopsis:
        continue
    movies_with_synopsis.append(movie_detail)

print('Movies with Plot Synopsis: ' + str(len(movies_with_synopsis)))

In [ ]:
with open('movies_with_synopsis.pkl', 'wb') as f:
    pickle.dump(movies_with_synopsis, f)    

In [5]:
movies_with_synopsis = pickle.load(open('movies_with_synopsis.pkl', 'rb'))

In [6]:
print(len(movies_with_synopsis))

1339


In [7]:
movie_detail = movies_with_synopsis[0]

In [8]:
import re
import string

lemmatizer = WordNetLemmatizer()

#for processing the text
def text_preprocessing(text):
    text = re.sub('\w*\d\w*', ' ', text) #match only letters
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text.lower()) #lowercase all text and remove punctuation
    word_list = nltk.word_tokenize(text)
    text = ' '.join([lemmatizer.lemmatize(word) for word in word_list]) #lemmatize text and recombined into one string
    return text

In [9]:
#for displaying the topics i.e. PCA components and words associated with them
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for idx, topic in enumerate(model.components_):
        if not topic_names or not topic_names[idx]:
            print("\nTopic ", idx)
        else:
            print("\nTopic: '",topic_names[idx],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))


In [10]:
#for finding reviews similar to the plot synopsis, the ultimate spoiler
def spoiler_similarity(Vt):
    similarities = []
    for i in range(Vt.shape[0]):
        similarity_matrix = cosine_similarity((Vt.iloc[0], Vt.iloc[i])) #input must be tuple of both 1D arrays to get matrix
        similarity  = similarity_matrix[0][1] #gives upper-right value in matrix i.e. how similar vectors are
        similarities.append(tuple((i, similarity))) #tuple needs double brackets around input
    return sorted(similarities, key = lambda x: x[1], reverse = True)[1:] #use this to check for most similar


In [11]:
#for use in converting gunning fog score to school level
def gunning_fog_to_school(score):
    if score > 0 and score <= 5:
        return 'elementary_school'
    elif score >= 6 and score <= 8:
        return 'middle_school'
    elif score >= 9 and score <= 12:
        return 'high_school'
    elif score >= 13 and score <= 17:
        return 'college'
    else:
        return 'other'

In [27]:
def contextual_spoiler_detection(movie_detail, model_name):
    synopsis = movie_detail['plot_synopsis']
    reviews = movie_detail['reviews_df']
    title = movie_detail['title']
    
    content = reviews['review_text'].values
    content = np.append(synopsis, content)
    
    stop_words = nltk.corpus.stopwords.words('english')

    new_stop_words = ['actor', 'actress', 'director', 'film', 
                      'movie', 'rating', 'spoiler', 'book']

    for word in title.split(' '):
        new_stop_words.append(word.lower())
    
    stop_words.extend(new_stop_words)
    
    content = [text_preprocessing(text) for text in content]
    
    vectorizer = CountVectorizer(ngram_range=(1,3), stop_words=stop_words, min_df=5)
    
    doc_word = vectorizer.fit_transform(content)
        
    if model_name == 'lda':
        num_components = 15 #seems like the optimal for LDA
        model = LatentDirichletAllocation(n_components=num_components, random_state=42, learning_method = 'batch')        
    elif model_name == 'lsa':
        num_components = 15 #seems like the optimal for LSA
        model = TruncatedSVD(num_components)
    elif model_name == 'nmf':
        num_components = 20 #seems like the optimal for NMF
        model = NMF(num_components)
        
    doc_topic = model.fit_transform(doc_word)
    
    
    component_names = ['Topic ' + str(i) for i in range(len(model.components_))]

    topic_word = pd.DataFrame(model.components_.round(3),
             index = component_names,
             columns = vectorizer.get_feature_names())
    
#     display_topics(model, vectorizer.get_feature_names(), 10)
    
    Vt = pd.DataFrame(doc_topic.round(5),
             index = content,
             columns = component_names)
    
    return spoiler_similarity(Vt)

In [29]:
review_df_list = []

for idx, movie_detail in enumerate(movies_with_synopsis):
    print(idx)
    similarities_lsa = contextual_spoiler_detection(movie_detail, 'lsa')
    similarities_lda = contextual_spoiler_detection(movie_detail, 'lda')

    reviews = movie_detail['reviews_df']
    truncated_reviews = reviews.reset_index().drop(columns = ['review_date', 'movie_id', 'is_spoiler', 'review_summary', 'rating'])
    truncated_reviews['cosine_similarity_lsa'] = [similarity[1] for similarity in sorted(similarities_lsa)]
    truncated_reviews['cosine_similarity_lda'] = [similarity[1] for similarity in sorted(similarities_lda)]
    truncated_reviews['flesch_reading_ease_score'] = truncated_reviews['review_text'].apply(lambda text: round(textstat.flesch_reading_ease(text)))
    truncated_reviews['difficult_words_score'] = truncated_reviews['review_text'].apply(lambda text: round(textstat.difficult_words(text)))
    truncated_reviews['linsear_write_score'] = truncated_reviews['review_text'].apply(lambda text: round(textstat.linsear_write_formula(text)))
    truncated_reviews['gunning_fog_score'] = truncated_reviews['review_text'].apply(lambda text: round(textstat.gunning_fog(text)))
           
    school_levels = truncated_reviews['gunning_fog_score'].apply(gunning_fog_to_school)
    school_df = pd.DataFrame(school_levels)
    school_df = pd.get_dummies(school_df)
    truncated_reviews = truncated_reviews.join(school_df, how = 'left').drop('gunning_fog_score', axis = 1)
    review_df_list.append(truncated_reviews)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [28]:
movie_detail = movies_with_synopsis[528]
contextual_spoiler_detection(movie_detail, 'lsa')

[(10, 0.5991405510354604),
 (1, 0.2866749453454609),
 (7, 0.2833159043080627),
 (6, 0.2603836175000564),
 (2, 0.23344746478316672),
 (4, 0.2269921989079538),
 (8, 0.20405153212141022),
 (9, 0.19922079907389897),
 (5, 0.1489527657187973),
 (3, 0.14482890667709386),
 (12, 0.06634803034198845),
 (11, 0.050557101217731076)]

In [30]:
combined = pd.concat(review_df_list, sort = False)
combined = combined.rename(columns={'gunning_fog_score_elementary_school': 'elementary_school', 
                          'gunning_fog_score_middle_school': 'middle_school', 
                          'gunning_fog_score_high_school': 'high_school', 
                          'gunning_fog_score_college':'college'})

combined = combined.drop('gunning_fog_score_other', axis = 1)
school_list = ['elementary_school', 'middle_school', 'high_school', 'college']
for school in school_list:
    combined[school] = combined[school].fillna(0)
    combined[school] = combined[school].astype(np.uint8)
combined

,index,user_id,review_text,cosine_similarity_lsa,cosine_similarity_lda,flesch_reading_ease_score,difficult_words_score,linsear_write_score,college,high_school,middle_school,elementary_school
0,504359,ur5986401,The Nutcracker has always been a somewhat prob...,0.286675,0.003068,33,52,33,0,0,0,0
1,504360,ur20293899,I first watched this when I was 15. I enjoyed ...,0.233447,0.003891,80,22,8,0,0,1,0
2,504361,ur6325108,BEWARE OF BOGUS REVIEWS & REVIEWERS. SOME REVI...,0.144829,0.007065,64,33,6,0,1,0,0
3,504362,ur0062676,Those who have given this production such a lo...,0.226992,0.001958,36,102,22,0,0,0,0
4,504363,ur1765736,I'd seen parts of this production before but I...,0.148953,0.003369,55,64,13,1,0,0,0
5,504364,ur2696700,"As far as I can recall, Balanchine's alteratio...",0.260384,0.004700,10,26,31,0,0,0,0
6,504365,ur20552756,This 1993 Balanchine version is not as good as...,0.283316,0.004038,60,60,11,0,1,0,0
7,504366,ur4313626,Both visually and musically stunning. A treat ...,0.204052,0.008408,46,37,9,1,0,0,0
8,504367,ur8036190,This movie is a perfect portrayal of The Nutcr...,0.199221,0.005446,76,20,12,0,1,0,0
9,504368,ur0437174,This is Warmer Brothers' film version of compo...,0.599141,0.002939,53,42,15,1,0,0,0


In [31]:
combined.to_pickle('review_stats.pkl')

In [151]:
# test_data = Vt.iloc[3].name
# print(test_data)
# print(textstat.flesch_reading_ease(test_data)) #how easy it is to read the text
# # print(textstat.smog_index(test_data)) #unrounded grade level, based on entire composition, assuming more than 3 sentences
# # print(textstat.flesch_kincaid_grade(test_data)) #unrounded grade level
# # print(textstat.coleman_liau_index(test_data)) #grade level of text
# # print(textstat.automated_readability_index(test_data)) #grade level required to understand text
print(textstat.dale_chall_readability_score(test_data)) #grade level where 10 is college
# print(textstat.difficult_words(test_data)) #how difficult words are
# print(textstat.linsear_write_formula(test_data)) #how difficult words are based on number of syllables
# print(textstat.gunning_fog(test_data)) #takes into account complex words
# # print(textstat.text_standard(test_data)) #gives you the grade level

7.5


In [ ]:
#seems like using cosine similarity against summary is not working that well

In [ ]:
spoiler_df = pd.read_pickle('./spoiler_df.pkl')

In [ ]:
spoilers = (spoiler_df['review_text'].apply(lambda x: re.sub('\w*\d\w*', ' ', x)
                                     .apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', text.lower()) )))

In [ ]:
corpus = spoilers

for text in corpus:
    text = re.sub('\w*\d\w*', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text.lower())

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
newStopWords = ['actor','director', 'film', 'movie', 'rating', 
                'spoiler']
stopwords.extend(newStopWords)


vectorizer = TfidfVectorizer(stop_words=stopwords,
                             token_pattern = '[a-zA-Z]+',
                             ngram_range = (1,2))
doc_word = vectorizer.fit_transform(corpus)
doc_word.shape

In [ ]:
spoilers = pd.read_pickle('./spoiler_df.pkl')

In [ ]:
spoilers['word_count'] = spoilers['review_text'].apply(lambda x: len(x.split(' ')))

In [ ]:
non_spoilers = pd.read_pickle('./non_spoiler_df.pkl')

In [ ]:
non_spoilers['word_count'] = non_spoilers['review_text'].apply(lambda x: len(x.split(' ')))

In [ ]:
non_spoiler_count = non_spoilers['word_count'].value_counts()
non_spoiler_lengths = non_spoiler_count.values
non_spoiler_freqs = non_spoiler_count.index.values

In [ ]:
spoiler_count = spoilers['word_count'].value_counts()
spoiler_lengths = spoiler_count.values
spoiler_freqs = spoiler_count.index.values

In [ ]:
plt.figure(figsize=(12,6));
sns.lineplot(non_spoiler_count, non_spoiler_freqs, color='g', ci=None);
sns.lineplot(spoiler_count, spoiler_freqs, color='r', ci=None);
plt.title('Spoiler and Non-Spoiler Review Word Count', y=1.05, fontsize=16);
plt.xlabel(xlabel = 'Word Count', fontsize = 12, x = 1.05);
plt.ylabel(ylabel = 'Frequency', fontsize = 12, rotation = 0, y=1.05);
plt.legend(labels=['Non-Spoiler', 'Spoiler']);
plt.savefig('review_word_count.png', bbox_inches = 'tight')

In [ ]:
substring = 'Star Wars'
star_wars_movie_details = []

for movie_detail in movie_details:
    title = movie_detail['title']
    if substring in title:
        star_wars_movie_details.append(movie_detail)

In [ ]:
star_wars_II = star_wars_movie_details[0]

In [ ]:
movie_details